## 1.加载keras模块

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input, VGG16
from keras import backend as K
import numpy as np

Using TensorFlow backend.
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framewo

### 加载预训练权值



### 定义新加入的layers

In [2]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':#返回默认的图像的维度顺序
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
# # 定义网络输入层shape
input_tensor = Input(shape=input_shape)
model = VGG16(weights='imagenet',#声明VGG网络，并指定加载imagenet预训练模型
              include_top=False,#不包含全连接层
              input_tensor=input_tensor)#向VGG网络追加一些全连接层和输出层

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(128, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))
#指定输出层为2个神经元做2分类，激活函数选用softmax

#把上述网络每一层layer拷贝至一个新的网络，形成一个整体网络
new_model = Sequential()
for l in model.layers:
    new_model.add(l)
new_model.add(top_model)
model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_____________________

### 设置不需微调的layers的trainable属性
并调用compile函数重新编译网络

In [3]:

from keras.optimizers import SGD
#在新的整体网络上设置对所有原VGG层均不参与微调
for layer in new_model.layers[:4]:
    layer.trainable = False

#优化器，随机梯度下降（学习率，动量参数）
#列表，包含评估模型在训练和测试时的性能的指标
new_model.compile(loss='categorical_crossentropy',#注意此处应选用何种loss function
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

### 定义ImageDataGenerator


In [6]:
train_data_dir = r'F:\QQPCmgr\Desktop\aitest2\dogs-vs-cats\train'#根据data路径
validation_data_dir = r'F:\QQPCmgr\Desktop\aitest2\dogs-vs-cats\validation'
nb_train_samples = 10835#用于训练的样本数
nb_validation_samples = 4000
epochs = 2  #1个epoch=训练集全部训练一次
batch_size = 20  #每次迭代 训练集/20=1个biath_size的大小



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255, #缩放
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)#布尔值，进行随机水平翻转


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,#目标目录
    target_size=(img_width, img_height),#所有图像调整为150*150
    batch_size=batch_size,
    class_mode='categorical')#决定返回的标签数组的形式，这里是返回2D的one-hot编码标签

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [7]:
#利用python的生成器，逐个生成数据的batch并进行训练
new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size, #当生成器返回这个参数次数据时计一个epoch结束，执行下一个epoch
    epochs=epochs,#数据迭代的轮数
    validation_data=validation_generator,#生成验证集的生成器
    validation_steps=nb_validation_samples // batch_size)    #当前一个参数为生成器时，本参数指定验证集的生成器返回次数

Epoch 1/2
541/541 [==============================] - 6881s 13s/step - loss: 0.2512 - acc: 0.9037 - val_loss: 0.2324 - val_acc: 0.9092
Epoch 2/2
541/541 [==============================] - 5143s 10s/step - loss: 0.1718 - acc: 0.9348 - val_loss: 0.1553 - val_acc: 0.9350


### 使用训练后模型预测图像





In [8]:
import cv2
img = cv2.resize(cv2.imread(r'F:\QQPCmgr\Desktop\aitest2\dogs-vs-cats\test\1.jpg'), (img_width, img_height)).astype(np.float32)
#使用imread读取rgb图片时，得到的array的shape是（高，宽，（B，G，R）
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)#扩展数组的形状，在x数组


#x = preprocess_input(x)

score = new_model.predict(x)


print(score)

[[0. 1.]]
